In [ ]:
import os
import pandas as pd
import hopsworks
import joblib
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from PIL import Image
from dotenv import load_dotenv
from src.paths import PARENT_DIR

In [ ]:
# HOPSWORKS_PROJECT_NAME = 'Vivekmaj1'

# # load key-value pairs from .env file located in the parent directory
# load_dotenv(PARENT_DIR / '.env')

# HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

project = hopsworks.login(
)

fs = project.get_feature_store()

In [ ]:
feature_view = fs.get_feature_view(name="iris", version=1)

In [ ]:
class ExtractExtraFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Create new features
        X['sepal_area'] = X['sepal_length'] * X['sepal_width']
        X['petal_area'] = X['petal_length'] * X['petal_width']
        X['sepal_petal_length_diff'] = X['sepal_length'] - X['petal_length']
        X['sepal_petal_width_diff'] = X['sepal_width'] - X['petal_width']
        X['sepal_length_to_width'] = X['sepal_length'] / X['sepal_width']
        X['petal_length_to_width'] = X['petal_length'] / X['petal_width']
        
        return X

In [ ]:
mr = project.get_model_registry()
model = mr.get_model("iris_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/iris_model.pkl")

In [ ]:
batch_data = feature_view.get_batch_data()

y_pred = model.predict(batch_data)

y_pred

In [ ]:
flower = y_pred[y_pred.size-1]
flower_img = "../assets/" + flower + ".png"
img = Image.open(flower_img)            

img.save("../assets/latest_iris.png")

In [ ]:
iris_fg = fs.get_feature_group(name="iris", version=1)
df = iris_fg.read()
df

In [ ]:
label = df.iloc[-1]["variety"]
label

In [ ]:
label_flower = "../assets/" + label + ".png"

img = Image.open(label_flower)            

img.save("../assets/actual_iris.png")

In [ ]:
monitor_fg = fs.get_or_create_feature_group(name="iris_predictions",
                                  version=1,
                                  primary_key=["datetime"],
                                  description="Iris flower Prediction/Outcome Monitoring"
                                 )

In [ ]:
print(label)

In [ ]:
now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

data = {
    'prediction': [flower],
    'label': [label],
    'datetime': [now],
}
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

In [ ]:
# history_df = monitor_fg.read()
# history_df